In [1]:
from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template

import plotly.express as px
import pandas as pd
import numpy as np

dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.SLATE, dbc_css])

load_figure_template("SLATE")

education = (
    pd.read_csv("../Data/states_all.csv")
    .iloc[:, 1:]
    .assign(
        Avg_Math = lambda x: x[["AVG_MATH_4_SCORE", "AVG_MATH_8_SCORE"]].mean(axis=1),
        Avg_Reading = lambda x: x[["AVG_READING_4_SCORE", "AVG_READING_8_SCORE"]].mean(axis=1),
    )
)

all_options = {
    "By Subject": [
        {"label": "Math", "value": "Avg_Math" },
        {"label": "Reading", "value": "Avg_Reading"}
    ],
    "By Grade and Subject": [
        {"label": "4th Grade Math", "value": "AVG_MATH_4_SCORE" },
        {"label": "8th Grade Math", "value": "AVG_MATH_8_SCORE" },
        {"label": "4th Grade Reading", "value": "AVG_READING_4_SCORE" },
        {"label": "8th Grade Reading", "value": "AVG_READING_8_SCORE" },
    ],
}

app.layout= html.Div([
    dcc.RadioItems(
        id="report-type-radio", 
        options=["By Subject", "By Grade and Subject"], 
        value="By Grade and Subject"
    ),
    dcc.RadioItems(id="metric-radio"),
    dcc.Graph(id="metric-bar")
])

@app.callback(
    Output("metric-radio", "options"), 
    Input("report-type-radio", "value"))
def set_metrics_options(selected_report):
    return all_options[selected_report]

@app.callback(
    Output("metric-bar", "figure"),
    Input("metric-radio", "value"))
def plot_bar(metric):
    if not metric:
        raise PreventUpdate
    figure = px.bar((
        education
        .groupby("STATE", as_index=False)
        .agg({metric: "mean"})
        .sort_values(metric, ascending=False)
        ),
        x="STATE",
        y=metric
    )
    return figure

app.run_server(debug=True, port=8171)

/Users/pocholo/Desktop/projects/Python_Data_Visualization_Plotly_and_Dash/venv/lib/python3.11/site-packages/dash/dash.py:538: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



Dash app running on http://127.0.0.1:8171/
